In [1]:
import numpy as np
import random
from math import *
import time

import copy

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR

In [2]:
torch.cuda.set_device(1)
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
# activation function
def activation(x):
    return x * torch.sigmoid(x) 

In [4]:
# build ResNet with three blocks
class Net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(Net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        self.layer1 = torch.nn.Linear(layer_width, layer_width)
        self.layer2 = torch.nn.Linear(layer_width, layer_width)
        self.layer3 = torch.nn.Linear(layer_width, layer_width)
        self.layer4 = torch.nn.Linear(layer_width, layer_width)
        self.layer5 = torch.nn.Linear(layer_width, layer_width)
        self.layer6 = torch.nn.Linear(layer_width, layer_width)
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        y = self.layer_in(x)
        y = y + activation(self.layer2(activation(self.layer1(y)))) # residual block 1
        y = y + activation(self.layer4(activation(self.layer3(y)))) # residual block 2
        y = y + activation(self.layer6(activation(self.layer5(y)))) # residual block 3
        output = self.layer_out(y)
        return output

In [5]:
dimension = 3

In [6]:
input_width,layer_width = dimension, 8

In [7]:
net = Net(input_width,layer_width).cuda() # network for u on gpu

In [8]:
# defination of exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1) # compute norm of x; dim = 1 - > sum by row
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [9]:
# defination of f(x)
def f(x):
    x_norm = torch.norm(x, dim = 1)
    f_temp = 1/4*pi**2*torch.sin(pi/2*(1-x_norm)) + 1/2*pi*(dimension-1)/x_norm*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])

In [10]:
# generate points by random
def generate_sample(data_size):
    i = 0
    x = torch.tensor([])
    while x.size()[0] < data_size:
        temp_x = 2*torch.rand(data_size, dimension) - 1
        index = torch.where(torch.norm(temp_x, dim = 1) < 1)
        temp_x = temp_x[index]
        x = torch.cat((x,temp_x),0)
    x = x[1:data_size, :]
    return x

In [11]:
def model(x):
    x_norm = torch.norm(x, p = 2, dim = 1) 
    return (1-x_norm**2).reshape([x.size()[0], 1]) * net(x)

In [12]:
# loss function to DRM by auto differential
def loss_function(x):
#     x = generate_sample(data_size).cuda()
#     x.requires_grad = True
    u_hat = model(x)
    grad_u_hat = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = torch.ones(u_hat.shape).cuda(), create_graph = True)
    grad_u_sq = ((grad_u_hat[0]**2).sum(1)).reshape([len(grad_u_hat[0]), 1])
    part = torch.sum(0.5 * grad_u_sq  - f(x) * u_hat)  / len(x)
    return part

In [13]:
data_size = 1000
x = generate_sample(data_size).cuda()
x.requires_grad = True

In [14]:
def relative_l2_error():
    data_size_temp = 500
    x = generate_sample(data_size_temp).cuda() 
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [15]:
optimizer = optim.Adam(net.parameters())

In [16]:
epoch = 5000
data_size = 1000
loss_record = np.zeros(epoch)
error_record = np.zeros(epoch)
time_start = time.time()
for i in range(epoch):
    optimizer.zero_grad()
    x = generate_sample(data_size).cuda()
    x.requires_grad = True
    loss = loss_function(x)
    loss_record[i] = float(loss)
    error = relative_l2_error()
    error_record[i] = float(error)
    np.save("DRM_loss_3d.npy", loss_record)
    np.save("DRM_error_3d.npy", error_record)
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", loss.detach())
        print("current error is: ", error.detach())
    if i == epoch - 1:
        # save model parameters
        torch.save(net.state_dict(), 'net_params_DRM.pkl')
        
    loss.backward()
    optimizer.step() 
    torch.cuda.empty_cache() # clear memory
    
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(-0.5462, device='cuda:1')
current error is:  tensor(0.6347, device='cuda:1')
current epoch is:  50
current loss is:  tensor(-1.0517, device='cuda:1')
current error is:  tensor(0.1249, device='cuda:1')
current epoch is:  100
current loss is:  tensor(-0.9070, device='cuda:1')
current error is:  tensor(0.0675, device='cuda:1')
current epoch is:  150
current loss is:  tensor(-0.9350, device='cuda:1')
current error is:  tensor(0.0503, device='cuda:1')
current epoch is:  200
current loss is:  tensor(-0.9763, device='cuda:1')
current error is:  tensor(0.0477, device='cuda:1')
current epoch is:  250
current loss is:  tensor(-1.0259, device='cuda:1')
current error is:  tensor(0.0532, device='cuda:1')
current epoch is:  300
current loss is:  tensor(-0.8722, device='cuda:1')
current error is:  tensor(0.0602, device='cuda:1')
current epoch is:  350
current loss is:  tensor(-1.0304, device='cuda:1')
current error is:  tensor(0.0475, device='cuda:1')
cur

current epoch is:  3300
current loss is:  tensor(-0.9884, device='cuda:1')
current error is:  tensor(0.0112, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-0.9881, device='cuda:1')
current error is:  tensor(0.0110, device='cuda:1')
current epoch is:  3400
current loss is:  tensor(-0.9816, device='cuda:1')
current error is:  tensor(0.0112, device='cuda:1')
current epoch is:  3450
current loss is:  tensor(-0.9245, device='cuda:1')
current error is:  tensor(0.0120, device='cuda:1')
current epoch is:  3500
current loss is:  tensor(-0.9669, device='cuda:1')
current error is:  tensor(0.0103, device='cuda:1')
current epoch is:  3550
current loss is:  tensor(-1.0580, device='cuda:1')
current error is:  tensor(0.0084, device='cuda:1')
current epoch is:  3600
current loss is:  tensor(-0.9230, device='cuda:1')
current error is:  tensor(0.0266, device='cuda:1')
current epoch is:  3650
current loss is:  tensor(-1.0997, device='cuda:1')
current error is:  tensor(0.0149, device='c